In [29]:
# Dependencies and Setup
import pandas as pd
import json
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error

In [95]:
schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2018/schedule.json")
schedule_response=schedule_response_obj.json()

In [97]:
games_response_count=schedule_response["league"]["standard"]

In [98]:
# Get elements
NBA_Schedule=[]
for item in schedule_response["league"]["standard"]:
    item_dict={}
    item_dict["gameId"] = item["gameId"]
    item_dict["startDateEastern"] = item["startDateEastern"]
    item_dict["hTeam"] = item["hTeam"]["teamId"]
    item_dict["vTeam"] = item["vTeam"]["teamId"]

    # Append List
    NBA_Schedule.append({"gameid":item_dict["gameId"],
                         "gamedate":item_dict["startDateEastern"],
                         "hteam":item_dict["hTeam"],
                         "vteam":item_dict["vTeam"]})

In [99]:
NBA_Schedule_DF_initial=pd.DataFrame(NBA_Schedule)
NBA_Schedule_DF_initial["gameid"]=NBA_Schedule_DF_initial["gameid"].astype(int)
NBA_Schedule_DF_initial["hteam"]=NBA_Schedule_DF_initial["hteam"].astype(int)
NBA_Schedule_DF_initial["vteam"]=NBA_Schedule_DF_initial["vteam"].astype(int)

NBA_Schedule_DF_initial["gamedate"]=pd.to_datetime(NBA_Schedule_DF_initial['gamedate'])

NBA_Schedule_DF_initial.head()

,gamedate,gameid,hteam,vteam
0,2018-09-28,11800001,1610612755,15016
1,2018-09-28,11800002,1610612766,1610612738
2,2018-09-29,11800003,1610612761,1610612757
3,2018-09-29,11800004,1610612742,15021
4,2018-09-29,11800005,1610612744,1610612750


In [100]:
NBA_Schedule_DF_initial.head()  
#October 16, 2018 Season start

,gamedate,gameid,hteam,vteam
0,2018-09-28,11800001,1610612755,15016
1,2018-09-28,11800002,1610612766,1610612738
2,2018-09-29,11800003,1610612761,1610612757
3,2018-09-29,11800004,1610612742,15021
4,2018-09-29,11800005,1610612744,1610612750


In [101]:
#Load home team_data
NBA_Schedule_DF_initial_1=NBA_Schedule_DF_initial[NBA_Schedule_DF_initial["gamedate"] >= '2018-10-16']
nba_cols = ["gamedate", "gameid", "hteam"]
NBA_Schedule_DF_initial_1a= NBA_Schedule_DF_initial_1[nba_cols].copy()
NBA_Schedule_DF_initial_1b = NBA_Schedule_DF_initial_1a.rename(columns={"hteam": "teamid"})
NBA_Schedule_DF_initial_1b['home_ind'] = 1

In [102]:
#Load visitor team_data
NBA_Schedule_DF_initial_2=NBA_Schedule_DF_initial[NBA_Schedule_DF_initial["gamedate"] >= '2018-10-16']
nba_cols = ["gamedate", "gameid", "vteam"]
NBA_Schedule_DF_initial_2a= NBA_Schedule_DF_initial_2[nba_cols].copy()
NBA_Schedule_DF_initial_2b = NBA_Schedule_DF_initial_2a.rename(columns={"vteam": "teamid"})
NBA_Schedule_DF_initial_2b['home_ind'] = 0

In [104]:
NBA_Schedule_DF = pd.concat([NBA_Schedule_DF_initial_1b,NBA_Schedule_DF_initial_2b])

In [30]:
engine = create_engine("sqlite:///db/NBA_Data.sqlite")
conn = engine.connect()

engine.table_names()

['Team_Locations', 'Team_Schedule', 'ranked_2016_2017']

In [106]:
# Set index
NBA_Schedule_DF.set_index("teamid", inplace=True)

# load final df into sqlite database
NBA_Schedule_DF.to_sql('Team_Schedule', conn, if_exists='replace', index=True)

In [25]:
pd.read_sql('create table Team_Schedule_Final as select a.*, b.teamFullName, b.teamURLcode, TRIM(b.city) as city_name, 
            b.conferenceName, b.divisionName from Team_Schedule a join NBA_Teams b on a.teamid=b.teamid', conn)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [26]:
pd.read_sql('select * from Team_Schedule_Final', conn)

,teamid,gamedate,gameid,home_ind,teamFullName,teamURLcode,city_name,conferenceName,divisionName
0,1610612738,2018-10-16 00:00:00.000000,21800001,1,Boston Celtics,celtics,Boston,East,Atlantic
1,1610612738,2018-10-22 00:00:00.000000,21800040,1,Boston Celtics,celtics,Boston,East,Atlantic
2,1610612738,2018-10-30 00:00:00.000000,21800099,1,Boston Celtics,celtics,Boston,East,Atlantic
3,1610612738,2018-11-01 00:00:00.000000,21800115,1,Boston Celtics,celtics,Boston,East,Atlantic
4,1610612738,2018-11-14 00:00:00.000000,21800204,1,Boston Celtics,celtics,Boston,East,Atlantic
5,1610612738,2018-11-16 00:00:00.000000,21800216,1,Boston Celtics,celtics,Boston,East,Atlantic
6,1610612738,2018-11-17 00:00:00.000000,21800229,1,Boston Celtics,celtics,Boston,East,Atlantic
7,1610612738,2018-11-21 00:00:00.000000,21800255,1,Boston Celtics,celtics,Boston,East,Atlantic
8,1610612738,2018-11-30 00:00:00.000000,21800319,1,Boston Celtics,celtics,Boston,East,Atlantic
9,1610612738,2018-12-06 00:00:00.000000,21800365,1,Boston Celtics,celtics,Boston,East,Atlantic


In [37]:
pd.read_sql('create table Team_Schedule_Final as select teamid, gamedate, gameid, home_ind, \
            case when home_ind=1 then teamFullName else null end as teamFullName, \
            case when home_ind=1 then teamURLcode else null end as teamURLcode, \
            case when home_ind=1 then city_name else null end as city_name, \
            case when home_ind=1 then conferenceName else null end as conferenceName, \
            case when home_ind=1 then divisionName else null end as divisionName \
            from Team_Schedule', conn)

ResourceClosedError: This result object does not return rows. It has been closed automatically.